In [57]:
import pandas as pd 
import numpy as np 
import requests
import json
import dotenv
import os
from bs4 import BeautifulSoup
import sys
sys.tracebacklimit = 0

In [2]:
dotenv.load_dotenv()
spotify_client_id = os.getenv('spotify_clientid')
spotify_client_secret = os.getenv('spotify_clientsecret')


url = 'https://accounts.spotify.com/api/token'
headers = {'Content-Type': 'application/x-www-form-urlencoded',}
data = {
    'grant_type': 'client_credentials',
    'client_id': spotify_client_id,
    'client_secret': spotify_client_secret,
}

response = requests.post(url, headers=headers, data=data)

token_info = response.json()
access_token = token_info['access_token']

In [3]:
artist_id = "4YRxDV8wJFPHPTeXepOstw" # Arijit Singh
# artist_id = "2ae6PxICSOZHvjqiCcgon8" # SPB
# artist_id = "2oSONSC9zQ4UonDKnLqksx" # Atif Aslam
# artist_id = "1Xyo4u8uXC1ZmMpatF05PJ" # The Weeknd


options = "appears_on"
limit = "50"
url = f'https://api.spotify.com/v1/artists/{artist_id}/albums?include_groups={options}&limit={limit}'
headers = {'Authorization': f'Bearer {access_token}'}
response = requests.get(url, headers=headers) 
total_album_no = json.loads(response.content)['total']

album_dict = {}
print(np.arange(0,total_album_no,50))
for i in np.arange(0,total_album_no,50):
    print(i)
    offset = i
    url = f'https://api.spotify.com/v1/artists/{artist_id}/albums?include_groups={options}&limit={limit}&offset={offset}'
    headers = {'Authorization': f'Bearer {access_token}'}
    response = requests.get(url, headers=headers) 

    df = pd.json_normalize(json.loads(response.text), record_path=['items']).drop("available_markets",axis=1)
    album_dict[i] = df
album_df = pd.concat(album_dict.values())


[   0   50  100  150  200  250  300  350  400  450  500  550  600  650
  700  750  800  850  900  950 1000 1050 1100 1150 1200]
0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100
1150
1200


In [7]:
response

<Response [429]>

In [4]:
album_df['Album_ID'] = album_df['external_urls.spotify'].apply(lambda x: x.split("/")[-1])
album_df['Producer'] = album_df['artists'].apply(lambda x: x[0]['name'])
album_df['Producer_ID'] = album_df['artists'].apply(lambda x: x[0]['id'])
album_df['name'] = album_df['name'].apply(lambda x: x.replace("(Original Motion Picture Soundtrack)",""))
album_df.to_csv("albums.csv")
album_df


,album_type,total_tracks,href,id,images,name,release_date,release_date_precision,type,uri,artists,album_group,external_urls.spotify,Album_ID,Producer,Producer_ID
0,album,12,https://api.spotify.com/v1/albums/0Rkv5iqjF2ue...,0Rkv5iqjF2uenfL0OVB8hg,[{'url': 'https://i.scdn.co/image/ab67616d0000...,Aashiqui 2,2013-04-06,day,album,spotify:album:0Rkv5iqjF2uenfL0OVB8hg,[{'external_urls': {'spotify': 'https://open.s...,appears_on,https://open.spotify.com/album/0Rkv5iqjF2uenfL...,0Rkv5iqjF2uenfL0OVB8hg,Mithoon,09UmIX92EUH9hAK4bxvHx6
1,album,9,https://api.spotify.com/v1/albums/2Lxoc72vRTGd...,2Lxoc72vRTGdQfMvj7Ovi1,[{'url': 'https://i.scdn.co/image/ab67616d0000...,Yeh Jawaani Hai Deewani,2013-03-30,day,album,spotify:album:2Lxoc72vRTGdQfMvj7Ovi1,[{'external_urls': {'spotify': 'https://open.s...,appears_on,https://open.spotify.com/album/2Lxoc72vRTGdQfM...,2Lxoc72vRTGdQfMvj7Ovi1,Pritam,1wRPtKGflJrBx9BmLsSwlU
2,album,8,https://api.spotify.com/v1/albums/0a183xiCHiC1...,0a183xiCHiC1GQd8ou7WXO,[{'url': 'https://i.scdn.co/image/ab67616d0000...,ANIMAL,2023-11-24,day,album,spotify:album:0a183xiCHiC1GQd8ou7WXO,[{'external_urls': {'spotify': 'https://open.s...,appears_on,https://open.spotify.com/album/0a183xiCHiC1GQd...,0a183xiCHiC1GQd8ou7WXO,Manan Bhardwaj,3pQ4aA7dkolyjUAMrVScgh
3,album,16,https://api.spotify.com/v1/albums/0LcyzKKw3RjF...,0LcyzKKw3RjFKL6ygISTeU,[{'url': 'https://i.scdn.co/image/ab67616d0000...,Ek Tha Raja,2024-03-18,day,album,spotify:album:0LcyzKKw3RjFKL6ygISTeU,[{'external_urls': {'spotify': 'https://open.s...,appears_on,https://open.spotify.com/album/0LcyzKKw3RjFKL6...,0LcyzKKw3RjFKL6ygISTeU,Badshah,0y59o4v8uw5crbN9M3JiL1
4,album,6,https://api.spotify.com/v1/albums/7sVzilmsqYcL...,7sVzilmsqYcLMYw2gtvoZM,[{'url': 'https://i.scdn.co/image/ab67616d0000...,Hamari Adhuri Kahani,2015-05-21,day,album,spotify:album:7sVzilmsqYcLMYw2gtvoZM,[{'external_urls': {'spotify': 'https://open.s...,appears_on,https://open.spotify.com/album/7sVzilmsqYcLMYw...,7sVzilmsqYcLMYw2gtvoZM,Jeet Gannguli,2kkQthS9OLpK4UqNWYqoVl
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42,single,1,https://api.spotify.com/v1/albums/33iD1j5tkqYg...,33iD1j5tkqYgeXRaV9AW0Z,[{'url': 'https://i.scdn.co/image/ab67616d0000...,Pal (Chillwave),2023-08-30,day,album,spotify:album:33iD1j5tkqYgeXRaV9AW0Z,[{'external_urls': {'spotify': 'https://open.s...,appears_on,https://open.spotify.com/album/33iD1j5tkqYgeXR...,33iD1j5tkqYgeXRaV9AW0Z,Various Artists,0LyfQWJT6nXafLPZqxe9Of
43,compilation,9,https://api.spotify.com/v1/albums/3FlOGXqUxZPW...,3FlOGXqUxZPWE6LJxykyer,[{'url': 'https://i.scdn.co/image/ab67616d0000...,"Open Stage Recreations, Vol. 93",2023-10-20,day,album,spotify:album:3FlOGXqUxZPWE6LJxykyer,[{'external_urls': {'spotify': 'https://open.s...,appears_on,https://open.spotify.com/album/3FlOGXqUxZPWE6L...,3FlOGXqUxZPWE6LJxykyer,Various Artists,0LyfQWJT6nXafLPZqxe9Of
44,compilation,7,https://api.spotify.com/v1/albums/0j6Qzm7zGk3Y...,0j6Qzm7zGk3YpqKbOqIolZ,[{'url': 'https://i.scdn.co/image/ab67616d0000...,Celebrating World Music Day With Indipop Chart...,2015-06-18,day,album,spotify:album:0j6Qzm7zGk3YpqKbOqIolZ,[{'external_urls': {'spotify': 'https://open.s...,appears_on,https://open.spotify.com/album/0j6Qzm7zGk3YpqK...,0j6Qzm7zGk3YpqKbOqIolZ,Various Artists,0LyfQWJT6nXafLPZqxe9Of
45,single,1,https://api.spotify.com/v1/albums/0eQbkiscxEdp...,0eQbkiscxEdpZefI3PhwbG,[{'url': 'https://i.scdn.co/image/ab67616d0000...,Aa Seetadevi Navvula (Lofi Flip),2024-06-27,day,album,spotify:album:0eQbkiscxEdpZefI3PhwbG,[{'external_urls': {'spotify': 'https://open.s...,appears_on,https://open.spotify.com/album/0eQbkiscxEdpZef...,0eQbkiscxEdpZefI3PhwbG,Various Artists,0LyfQWJT6nXafLPZqxe9Of


In [5]:
import time
all_albums = {}

for i,album_id in enumerate(album_df['Album_ID'].unique()):
    print(i,len(album_df))
    url = f'https://api.spotify.com/v1/albums/{album_id}/tracks'
    try:
        response = requests.get(url, headers=headers) 
        df_t = pd.json_normalize(json.loads(response.text), record_path=['items']).drop("available_markets",axis=1)
    except:
        
        print("Sleeping for 3 seconds")
        time.sleep(3)
        response = requests.get(url, headers=headers) 
        df_t = pd.json_normalize(json.loads(response.text), record_path=['items']).drop("available_markets",axis=1)


    df_t['Album_ID'] = album_id



    df_t['Artist Song'] = df_t['artists'].apply(lambda x: artist_id in str(x))
    all_albums[album_id] = df_t

df_tracks = pd.concat(all_albums.values())
df_tracks.to_csv("tracks.csv")

0 1247
1 1247
2 1247
3 1247
4 1247
5 1247
6 1247
7 1247
8 1247
9 1247
10 1247
11 1247
12 1247
13 1247
14 1247
15 1247
16 1247
17 1247
18 1247
19 1247
20 1247
21 1247
22 1247
23 1247
24 1247
25 1247
26 1247
27 1247
28 1247
29 1247
30 1247
31 1247
32 1247
33 1247
34 1247
35 1247
36 1247
37 1247
38 1247
39 1247
40 1247
41 1247
42 1247
43 1247
44 1247
45 1247
46 1247
47 1247
48 1247
49 1247
50 1247
51 1247
52 1247
53 1247
54 1247
55 1247
56 1247
57 1247
58 1247
59 1247
60 1247
61 1247
62 1247
63 1247
64 1247
65 1247
66 1247
67 1247
68 1247
69 1247
70 1247
71 1247
72 1247
73 1247
74 1247
75 1247
76 1247
77 1247
78 1247
79 1247
80 1247
81 1247
82 1247
83 1247
84 1247
85 1247
86 1247
87 1247
88 1247
89 1247
90 1247
91 1247
92 1247
93 1247
94 1247
95 1247
96 1247
97 1247
98 1247
99 1247
100 1247
101 1247
102 1247
103 1247
104 1247
105 1247
106 1247
107 1247
108 1247
109 1247
110 1247
111 1247
112 1247
113 1247
114 1247
115 1247
116 1247
117 1247
118 1247
119 1247
120 1247
121 1247
122 1247
123

In [6]:
json.loads(response.text)

{'href': 'https://api.spotify.com/v1/albums/5pLmK7ur4utfjjhWEHbpCs/tracks?offset=0&limit=20',
 'items': [{'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/1kl3hTsjxtAQro0wpNVf96'},
     'href': 'https://api.spotify.com/v1/artists/1kl3hTsjxtAQro0wpNVf96',
     'id': '1kl3hTsjxtAQro0wpNVf96',
     'name': 'Surojit Chatterjee',
     'type': 'artist',
     'uri': 'spotify:artist:1kl3hTsjxtAQro0wpNVf96'},
    {'external_urls': {'spotify': 'https://open.spotify.com/artist/7gjiYwM6O5sNuYBaCdpCXA'},
     'href': 'https://api.spotify.com/v1/artists/7gjiYwM6O5sNuYBaCdpCXA',
     'id': '7gjiYwM6O5sNuYBaCdpCXA',
     'name': 'Iman Chakraborty',
     'type': 'artist',
     'uri': 'spotify:artist:7gjiYwM6O5sNuYBaCdpCXA'}],
   'available_markets': ['AR',
    'AU',
    'AT',
    'BE',
    'BO',
    'BR',
    'BG',
    'CA',
    'CL',
    'CO',
    'CR',
    'CY',
    'CZ',
    'DK',
    'DO',
    'DE',
    'EC',
    'EE',
    'SV',
    'FI',
    'FR',
    'GR',
    'GT',
    

In [7]:
artist_songs = df_tracks[df_tracks['Artist Song']==True]
artist_songs = artist_songs[['artists','duration_ms','id','name','Album_ID']].rename(columns = {'name':"Track_Name",
                                                                                                'artists':"Track_Artists"})

album_details = album_df[['total_tracks','name','release_date','artists','Album_ID','Producer','Producer_ID']].rename(columns = {'name':'Album_Name',
                                                                                                                                'artists':"Album_Artists"})

final = artist_songs.merge(album_details, on = "Album_ID", how = "left", indicator=True)
print(final._merge.value_counts())

def get_track_pop(track_id):
    url = f'https://api.spotify.com/v1/tracks/{track_id}'
    try: return json.loads(requests.get(url, headers=headers).text)['popularity']
    except: return np.nan

final['Popularity'] = final['id'].apply(get_track_pop)
final.to_csv("All_Details_Full.csv", index = False)
final

_merge
both          2095
left_only        0
right_only       0
Name: count, dtype: int64


,Track_Artists,duration_ms,id,Track_Name,Album_ID,total_tracks,Album_Name,release_date,Album_Artists,Producer,Producer_ID,_merge,Popularity
0,[{'external_urls': {'spotify': 'https://open.s...,261974,56zZ48jdyY2oDXHVnwg5Di,Tum Hi Ho,0Rkv5iqjF2uenfL0OVB8hg,12,Aashiqui 2,2013-04-06,[{'external_urls': {'spotify': 'https://open.s...,Mithoon,09UmIX92EUH9hAK4bxvHx6,both,74
1,[{'external_urls': {'spotify': 'https://open.s...,304899,6UgcN95w7vQxkR8sEFmwHG,Chahun Main Ya Naa,0Rkv5iqjF2uenfL0OVB8hg,12,Aashiqui 2,2013-04-06,[{'external_urls': {'spotify': 'https://open.s...,Mithoon,09UmIX92EUH9hAK4bxvHx6,both,68
2,[{'external_urls': {'spotify': 'https://open.s...,306511,55eUtE9v3uatR0y0NijM6N,Hum Mar Jayenge,0Rkv5iqjF2uenfL0OVB8hg,12,Aashiqui 2,2013-04-06,[{'external_urls': {'spotify': 'https://open.s...,Mithoon,09UmIX92EUH9hAK4bxvHx6,both,60
3,[{'external_urls': {'spotify': 'https://open.s...,266069,3E9ryvrKP3hUWX79PAakH2,Meri Aashiqui,0Rkv5iqjF2uenfL0OVB8hg,12,Aashiqui 2,2013-04-06,[{'external_urls': {'spotify': 'https://open.s...,Mithoon,09UmIX92EUH9hAK4bxvHx6,both,60
4,[{'external_urls': {'spotify': 'https://open.s...,214470,06zJUeKU5KFy50Cho6pDcr,Aasan Nahin Yahan,0Rkv5iqjF2uenfL0OVB8hg,12,Aashiqui 2,2013-04-06,[{'external_urls': {'spotify': 'https://open.s...,Mithoon,09UmIX92EUH9hAK4bxvHx6,both,66
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2090,[{'external_urls': {'spotify': 'https://open.s...,201731,5Q48TjFIQmxNDRFr6mNFuP,Pal - Chillwave,33iD1j5tkqYgeXRaV9AW0Z,1,Pal (Chillwave),2023-08-30,[{'external_urls': {'spotify': 'https://open.s...,Various Artists,0LyfQWJT6nXafLPZqxe9Of,both,0
2091,[{'external_urls': {'spotify': 'https://open.s...,253143,0IlyMpey6ixhj3I1EUYZy5,What Jhumka - Remix,3FlOGXqUxZPWE6LJxykyer,9,"Open Stage Recreations, Vol. 93",2023-10-20,[{'external_urls': {'spotify': 'https://open.s...,Various Artists,0LyfQWJT6nXafLPZqxe9Of,both,0
2092,[{'external_urls': {'spotify': 'https://open.s...,317413,7ERaNw1bhGEEaMc5CM9c0E,Rishton Ke Saare Manzar,0j6Qzm7zGk3YpqKbOqIolZ,7,Celebrating World Music Day With Indipop Chart...,2015-06-18,[{'external_urls': {'spotify': 'https://open.s...,Various Artists,0LyfQWJT6nXafLPZqxe9Of,both,0
2093,[{'external_urls': {'spotify': 'https://open.s...,85388,2W4MoVMmaptaBWdIq5u20B,Aa Seetadevi Navvula - Lofi Flip,0eQbkiscxEdpZefI3PhwbG,1,Aa Seetadevi Navvula (Lofi Flip),2024-06-27,[{'external_urls': {'spotify': 'https://open.s...,Various Artists,0LyfQWJT6nXafLPZqxe9Of,both,0


In [9]:
final_graph = final.copy()
final_graph = final_graph[final_graph['Producer']!="Various Artists"]
final_graph = final_graph[final_graph['Album_Name'].apply(lambda x: "Best" not in x.title())]
final_graph = final_graph[final_graph['Album_Name'].apply(lambda x: "Top" not in x.title())]
final_graph = final_graph[final_graph['Album_Name'].apply(lambda x: "Hits" not in x.title())]

final_graph = final_graph[final_graph['Album_Name'].apply(lambda x: "Remix" not in x.title())]
final_graph = final_graph[final_graph['Track_Name'].apply(lambda x: "Remix" not in x.title())]
final_graph = final_graph[final_graph['Track_Name'].apply(lambda x: "Mix" not in x.title())]
print(final_graph)
final_graph.to_csv("All_Details.csv", index = False)


                                          Track_Artists  duration_ms  \
0     [{'external_urls': {'spotify': 'https://open.s...       261974   
1     [{'external_urls': {'spotify': 'https://open.s...       304899   
2     [{'external_urls': {'spotify': 'https://open.s...       306511   
3     [{'external_urls': {'spotify': 'https://open.s...       266069   
4     [{'external_urls': {'spotify': 'https://open.s...       214470   
...                                                 ...          ...   
2005  [{'external_urls': {'spotify': 'https://open.s...       368758   
2033  [{'external_urls': {'spotify': 'https://open.s...       188813   
2043  [{'external_urls': {'spotify': 'https://open.s...       219193   
2060  [{'external_urls': {'spotify': 'https://open.s...       210619   
2062  [{'external_urls': {'spotify': 'https://open.s...       192400   

                          id                            Track_Name  \
0     56zZ48jdyY2oDXHVnwg5Di                             Tum Hi H

In [39]:
import lyricsgenius
from indic_transliteration import sanscript
from indic_transliteration.sanscript import SchemeMap, SCHEMES, transliterate
import time

genius_access_token = os.getenv('geniusclientaccesstoken')
genius = lyricsgenius.Genius(genius_access_token)
genius_ct = os.getenv('geniusclientsecret')
final_graph = pd.read_csv("All_Details.csv")

In [97]:
# first_n = 1200
# for i in range(final_graph.shape[0]):
#     if i==first_n: break
#     print(i)
#     song_dets = final_graph.iloc[i,:].to_dict()
#     # print(song_dets.keys())
#     lyrics = ""
#     query = song_dets['Track_Name'] + " "
#     try:
#         try:
#             lyrics = genius.search_song(query + " Romanized").lyrics
#         except:
#             lyrics = genius.search_song(query).lyrics
#             lyrics = transliterate(lyrics, sanscript.DEVANAGARI, sanscript.ITRANS)
#     except: continue
        
#     final_graph.loc[i,"Lyrics"] = lyrics.split("Lyrics")[-1].title()
#     time.sleep(1)
# final_graph = final_graph.iloc[:first_n, :]

def get_lyrics(title):
    search_url = "https://api.genius.com/search"
    headers = {"Authorization": f"Bearer {genius_access_token}"}
    params = {"q": title}
    response = requests.get(search_url, headers=headers, params=params)

    if len(response.json()["response"]["hits"])==0: return np.nan
    song_url = response.json()["response"]["hits"][0]["result"]["url"]

    page_response = requests.get(song_url)
    soup = BeautifulSoup(page_response.text, "html.parser")
    lyrics_div = soup.find("div", {"data-lyrics-container": "true"})
    try:
        lyrics_div = lyrics_div.get_text(separator="\n", strip=True)
    except: 
        return np.nan

    return lyrics_div

# final_graph['Lyrics'] = final_graph.apply(lambda x: get_lyrics(x['Track_Name'] + " Arijit Singh"), axis=1)
final_graph = final_graph.reset_index(drop=True)
for i in range(final_graph.shape[0]):
    x = final_graph.iloc[i,:].to_dict()
    lyric = get_lyrics(x['Track_Name'] + " Arijit Singh")
    lyr = str(lyric).split("\n")[3] if lyric==lyric else np.nan
    print(f"{i+1}) {x['Track_Name']} - {lyr}")

    final_graph.loc[i,"Lyrics"] = lyric



1) Tum Hi Ho - हम तेरे बिन अब रह नहीं सकते
2) Chahun Main Ya Naa - तू ही ये मुझको बता दे
3) Hum Mar Jayenge - काश तू मेरी आँखें भर दे
4) Meri Aashiqui - [Verse]
5) Aasan Nahin Yahan - आशिक़ को मिलती हैं ग़म की सौगातें
6) Milne Hai Mujhse Aayi - ओ, खुद से है या खुदा से इस पल मेरी लड़ाई
7) Ilahi - [Pre-Chorus]
8) Dilliwaali Girlfriend - What a नक़्श! What a नैन! दिल तेरा हो गया fan
9) Kabira (Encore) - दुआ में मीठी गुड़धानी ले गई
10) Satranga - [Verse 1]
11) Soulmate - तेरा दिल दुखा दे जो, ऐसी ना बात करूँ
12) Hamari Adhuri Kahani (Title Track) - [Verse 1]
13) Tujhe Kitna Chahne Lage - खुद को मुझे तू सौंप दे
14) Bekhayali (Arijit Singh Version) - तेरी नज़दीकियों की ख़ुशी बेहिसाब थी
15) Ae Dil Hai Mushkil Title Track - तू मेरा ख़ुदा, तू ही दुआ में शामिल
16) Channa Mereya - अच्छा चलता हूँ, दुआओं में याद रखना
17) The Breakup Song - साढ़े चार minute long
18) Alizeh - सच कहूँ तो, तुझसे ज्यादा
19) Bulleya - Reprise - ਵੇ ਕੀ ਕਰਾਂ? ਵੇ ਕੀ ਕਰਾਂ?
20) Channa Mereya - Unplugged - क़िस्से हमारे नज़दीकियों के
21) 

In [99]:
final_graph.sort_values("Popularity", ascending=False).to_csv("All_Details_with_Lyrics.csv", index = False)
# genius.search_song("9580441")
# genius.search_artist("arijit singh")

In [16]:
### TOP TRACKS OF ARTIST
artist_id = "4YRxDV8wJFPHPTeXepOstw" # Arijit Singh
# artist_id = "2ae6PxICSOZHvjqiCcgon8" # SPB
# artist_id = "2oSONSC9zQ4UonDKnLqksx" # Atif Aslam
# artist_id = "1Xyo4u8uXC1ZmMpatF05PJ" # The Weeknd

location = "IN"
url = f'https://api.spotify.com/v1/artists/{artist_id}/top-tracks?market={location}'
headers = {'Authorization': f'Bearer {access_token}'}
response = requests.get(url, headers=headers) 
top_tracks = json.loads(response.content)

# album_dict = {}
# print(np.arange(0,total_album_no,50))
# for i in np.arange(0,total_album_no,50):
#     print(i)
#     offset = i
#     url = f'https://api.spotify.com/v1/artists/{artist_id}/albums?include_groups={options}&limit={limit}&offset={offset}'
#     headers = {'Authorization': f'Bearer {access_token}'}
#     response = requests.get(url, headers=headers) 

#     df = pd.json_normalize(json.loads(response.text), record_path=['items']).drop("available_markets",axis=1)
#     album_dict[i] = df
# album_df = pd.concat(album_dict.values())

In [21]:
top_tracks['tracks']

[{'album': {'album_type': 'single',
   'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/3PpjqSKBer3obW6lDflOBx'},
     'href': 'https://api.spotify.com/v1/artists/3PpjqSKBer3obW6lDflOBx',
     'id': '3PpjqSKBer3obW6lDflOBx',
     'name': 'Ram Sampath',
     'type': 'artist',
     'uri': 'spotify:artist:3PpjqSKBer3obW6lDflOBx'},
    {'external_urls': {'spotify': 'https://open.spotify.com/artist/4YRxDV8wJFPHPTeXepOstw'},
     'href': 'https://api.spotify.com/v1/artists/4YRxDV8wJFPHPTeXepOstw',
     'id': '4YRxDV8wJFPHPTeXepOstw',
     'name': 'Arijit Singh',
     'type': 'artist',
     'uri': 'spotify:artist:4YRxDV8wJFPHPTeXepOstw'},
    {'external_urls': {'spotify': 'https://open.spotify.com/artist/7vCsTTACb2iT4fvBMnwp5w'},
     'href': 'https://api.spotify.com/v1/artists/7vCsTTACb2iT4fvBMnwp5w',
     'id': '7vCsTTACb2iT4fvBMnwp5w',
     'name': 'Prashant Pandey',
     'type': 'artist',
     'uri': 'spotify:artist:7vCsTTACb2iT4fvBMnwp5w'}],
   'external_urls': 